In [1]:
import pandas as pd
import numpy as np

import os, sys
from copy import deepcopy

import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

from itertools import combinations
import networkx as nx

os.environ['DGLBACKEND'] = 'pytorch'
import dgl
from dgl import function as fn

Using backend: pytorch


In [14]:
from dgl.nn import GraphConv, HeteroGraphConv  # type: ignore 

## dgl.heterograph(data_dict, num_nodes_dict=None, idtype=None, device=None)
Create a heterogeneous graph and return.

Parameters
data_dict (graph data) –

The dictionary data for constructing a heterogeneous graph. The keys are in the form of string triplets (src_type, edge_type, dst_type), specifying the source node, edge, and destination node types. The values are graph data in the form of (U,V), where (U[i],V[i]) forms the edge with ID i. The allowed graph data formats are:

In [24]:
# Case 2: Unidirectional bipartite graph
u = [0, 1, 0, 0, 1]
v = [0, 1, 2, 3, 2]

w=[1,2,3,4]
z=[0,1,1,0]

g = dgl.heterograph({('_U', '_E', '_V') : (u, v),
                     ('_V','_Z','_U'): (v,u)})  # type: ignore
u_fea = th.rand(2, 5)
v_fea = th.rand(4, 5)

In [37]:
# Case 2: Unidirectional bipartite graph
u = [0, 1, 0, 0, 1,0]
v = [0, 1, 2, 3, 2,4]


g = dgl.heterograph({('_U', '_E', '_V') : (u, v),
                     })  # type: ignore
u_fea = th.rand(2, 5)
v_fea = th.rand(5, 5)

In [30]:
g.etypes

['_E']

In [38]:
conv =  GraphConv(5,2)
conv(g, (u_fea,v_fea))

tensor([[-0.1303, -0.2852],
        [-0.2618, -0.4289],
        [-0.2773, -0.5049],
        [-0.1303, -0.2852],
        [-0.1303, -0.2852]], grad_fn=<AddBackward0>)

In [25]:
conv = HeteroGraphConv({
    '_E' : GraphConv(5,2),
    '_Z' : GraphConv(5,3),
    },
    aggregate='sum')

In [39]:
conv.weight

Parameter containing:
tensor([[ 0.7445, -0.1248],
        [-0.6177, -0.1996],
        [-0.5875, -0.7300],
        [ 0.5890, -0.4935],
        [ 0.2065,  0.3837]], requires_grad=True)

In [26]:
conv(g, {'_U': u_fea,'_V':v_fea})

{'_U': tensor([[-1.5566, -0.3702,  1.6320],
         [-1.3720,  0.0749,  1.3791]], grad_fn=<SumBackward1>),
 '_V': tensor([[ 0.1884,  0.1167],
         [ 0.3374, -0.0058],
         [ 0.3718,  0.0784],
         [ 0.1884,  0.1167]], grad_fn=<SumBackward1>)}

In [28]:
conv(g, {'_U': u_fea,'_V':v_fea})

KeyError: '_U'